In [1]:
# Testing the AddEdge Class in Stanford CoreNLP's Ssurgeon Library
# Stanford CoreNLP is a suite of natural language processing tools that provide functionalities such as tokenization, part-of-speech tagging, parsing, and more.
# Ssurgeon (Semantic Surgeon) is a component within Stanford CoreNLP that allows users to perform complex manipulations on the dependency graphs generated by the parser.
# MergeNode is a class within Ssurgeon designed to combines two words into one word
# This requires one of the nodes to be the head of a phrase of the words, and the dependent words can't have any extra edges in or out of that subgraph
# The word and lemma will be the combination of the words, squished together. Before and after will be updated to use the before and after of the endpoints of the subgraph

In [ ]:
# Two versions of the code are provided:
# First Version (SsurgeonMergeNodesComparison.java): Attempts to use Ssurgeon and compare it with manual merge.
# Second Version (SsurgeonMergeNodes.java): Introduces a variation by adding a POS tag as an attributes to guide the merge operation.

In [2]:
# Install Java
!apt-get update
!apt-get install -y openjdk-11-jdk-headless
!java -version

# Download Stanford CoreNLP
# Download Stanford CoreNLP 4.5.5
!wget https://nlp.stanford.edu/software/stanford-corenlp-4.5.5.zip
!unzip stanford-corenlp-4.5.5.zip

Get:1 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1227 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Get:4 https://packages.cloud.google.com/apt cloud-sdk InRelease [1618 B]       
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:6 https://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [28.6 kB]
Get:7 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [3339 kB]
Get:8 https://packages.cloud.google.com/apt cloud-sdk/main all Packages [1552 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]     
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [4090 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4532 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1274 kB]
Get:

In [3]:
!wget http://nlp.stanford.edu/software/stanford-corenlp-4.5.5-models-english.jar

--2024-10-17 15:41:26--  http://nlp.stanford.edu/software/stanford-corenlp-4.5.5-models-english.jar
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-4.5.5-models-english.jar [following]
--2024-10-17 15:41:26--  https://nlp.stanford.edu/software/stanford-corenlp-4.5.5-models-english.jar
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-4.5.5-models-english.jar [following]
--2024-10-17 15:41:26--  https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-4.5.5-models-english.jar
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|

In [4]:
!export CLASSPATH=$CLASSPATH:/kaggle/working/stanford-corenlp-4.5.5/*:/kaggle/working/stanford-corenlp-4.5.5-models-english.jar

In [5]:
import subprocess

try:
    result = subprocess.run(["jar", "tf", "/kaggle/working/stanford-corenlp-4.5.5/stanford-corenlp-4.5.5.jar"], 
                            capture_output=True, text=True, check=True)
    merge_nodes_entries = [line for line in result.stdout.split('\n') if 'MergeNodes' in line]
    
    if merge_nodes_entries:
        print("MergeNodes entries found:")
        for entry in merge_nodes_entries:
            print(entry)
    else:
        print("No MergeNodes entries found in the JAR file.")
except subprocess.CalledProcessError as e:
    print(f"An error occurred while executing the jar command: {e}")
except FileNotFoundError:
    print("The jar command was not found. Make sure Java is installed and in your PATH.")

MergeNodes entries found:
edu/stanford/nlp/semgraph/semgrex/ssurgeon/MergeNodes.class


In [ ]:
# Merge two words ("ice" and "cream") into a single compound noun ("icecream") within the dependency parse graph of the sentence: "The child likes ice cream."
# The program employs two methods: Ssurgeon MergeNodes and Manual Merge

In [ ]:
# Attempts to merge "ice" and "cream" using Ssurgeon, alongside a manual merge for comparison.

In [ ]:
# Test merging nodes using Ssurgeon
# Adds a MergeNodes edit to the SsurgeonPattern without specifying additional attributes
# Uses the execute method on the SsurgeonPattern
# Takes the entire semantic graph as a parameter
# Returns a Collection of SemanticGraph objects
# Checks if the results collection is not empty, creates a new IndexedWord representing the expected merged node, verifies if this expected merged node exists in the resulting graph

In [ ]:
# Semgrex pattern:
# {}=gov: Matches any word in the sentence and labels it as the governor (gov). In the sentence "The child likes ice cream.", this would match the verb "likes"
# >obj: Indicates that the governor (gov, i.e., "likes") has an object relation (obj) to another word
# ({word:" + nodesToMerge.get(1) + "}=node1: Retrieves the second element from the nodesToMerge list, matches a word that's the object of the verb, and labels this matched word as "node1
# >compound {word:ice}=node2: Retrieves the first element from the nodesToMerge list, matches a word that's in a compound relationship with node1, and labels this matched word as "node1

In [6]:
import subprocess
import os

java_code = '''

import edu.stanford.nlp.pipeline.*;
import edu.stanford.nlp.semgraph.*;
import edu.stanford.nlp.semgraph.semgrex.*;
import edu.stanford.nlp.ling.*;
import edu.stanford.nlp.util.*;
import edu.stanford.nlp.trees.GrammaticalRelation;
import edu.stanford.nlp.semgraph.semgrex.ssurgeon.*;

import java.util.*;

public class SsurgeonMergeNodesComparison {

    public static void main(String[] args) {
        Properties props = new Properties();
        props.setProperty("annotators", "tokenize,ssplit,pos,lemma,depparse");
        StanfordCoreNLP pipeline = new StanfordCoreNLP(props);
        
        // Sentence where "ice cream" forms a compound noun
        String text = "The child likes ice cream.";
        
        CoreDocument document = new CoreDocument(text);
        pipeline.annotate(document);
        
        SemanticGraph originalGraph = document.sentences().get(0).dependencyParse();
        
        System.out.println("Original Dependency Graph:");
        System.out.println(originalGraph.toString(SemanticGraph.OutputFormat.LIST));

        List<String> nodesToMerge = Arrays.asList("ice", "cream");
        
        try {
            // Clone the graph for both manual and Ssurgeon tests
            SemanticGraph ssurgeonGraph = new SemanticGraph(originalGraph);
            SemanticGraph manualGraph = new SemanticGraph(originalGraph);

            testSsurgeonMergeNodes(ssurgeonGraph, nodesToMerge);
            testManualMerge(manualGraph, nodesToMerge);
            
            System.out.println("Comparing results:");
            System.out.println("Ssurgeon MergeNodes result:");
            System.out.println(ssurgeonGraph.toString(SemanticGraph.OutputFormat.LIST));
            System.out.println("Manual merge result:");
            System.out.println(manualGraph.toString(SemanticGraph.OutputFormat.LIST));
            
        } catch (Exception e) {
            System.out.println("An error occurred: " + e.getMessage());
            e.printStackTrace();
        }
    }
    
    // Test merging nodes using Ssurgeon
    
    private static void testSsurgeonMergeNodes(SemanticGraph graph, List<String> nodesToMerge) {
    System.out.println("Testing Ssurgeon MergeNodes:");
    
    // Print the initial state of the graph before applying the merge
    System.out.println("Graph before applying Ssurgeon MergeNodes:");
    System.out.println(graph.toString(SemanticGraph.OutputFormat.LIST));

    // Create the Semgrex pattern to match the nodes
    String patternString = "{}=gov >obj ({word:" + nodesToMerge.get(1) + "}=node1 >compound {word:" + nodesToMerge.get(0) + "}=node2)";
    System.out.println("Semgrex Pattern: " + patternString);
    SemgrexPattern semgrexPattern = SemgrexPattern.compile(patternString);
    
    SemgrexMatcher matcher = semgrexPattern.matcher(graph);

    // Check if the pattern is found and log the nodes involved
    if (matcher.find()) {
        IndexedWord gov = matcher.getNode("gov");
        IndexedWord node1 = matcher.getNode("node1");  // "cream"
        IndexedWord node2 = matcher.getNode("node2");  // "ice"
        
        System.out.println("Match found:");
        System.out.println("Governor node: " + gov);
        System.out.println("Node1 (cream): " + node1);
        System.out.println("Node2 (ice): " + node2);
        
        // Log the state of the graph before the merge
        System.out.println("Graph state before Ssurgeon merge:");
        System.out.println(graph.toString(SemanticGraph.OutputFormat.LIST));

        // Create the Ssurgeon edit for merging nodes
        SsurgeonPattern surgeonPattern = new SsurgeonPattern(semgrexPattern);
        SsurgeonEdit mergeNodesEdit = new MergeNodes(Arrays.asList(node1.word(), node2.word()), new HashMap<>());
        surgeonPattern.addEdit(mergeNodesEdit);
        
        // Apply the Ssurgeon merge operation
        Collection<SemanticGraph> results = surgeonPattern.execute(graph);
        
        // Log the state of the graph immediately after applying the merge
        System.out.println("Graph after applying Ssurgeon MergeNodes (before cleanup):");
        System.out.println(graph.toString(SemanticGraph.OutputFormat.LIST));
        
        // Check if the graph was updated
        if (results.isEmpty()) {
            System.out.println("Ssurgeon MergeNodes failed to apply.");
        } else {
            graph = results.iterator().next();
            System.out.println("Ssurgeon MergeNodes applied successfully.");
        }

        // Verify if the merged node is present in the graph
        IndexedWord mergedNode = new IndexedWord(node1);
        mergedNode.setWord(node1.word() + node2.word());
        System.out.println("Merged node: " + mergedNode);

        if (graph.containsVertex(mergedNode)) {
            System.out.println("Merged node '" + mergedNode.word() + "' is present in the graph.");
        } else {
            System.out.println("Merged node '" + mergedNode.word() + "' is missing.");
        }

        // Log the final state of the graph
        System.out.println("Final graph after Ssurgeon merge and post-processing:");
        System.out.println(graph.toString(SemanticGraph.OutputFormat.LIST));
    } else {
        System.out.println("No match found for the pattern in Ssurgeon.");
    }
}   
    
    // Test manual merge of nodes
    
    private static void testManualMerge(SemanticGraph graph, List<String> nodesToMerge) {
    System.out.println("Testing Manual Merge:");
    String patternString = "{}=gov >obj ({word:" + nodesToMerge.get(1) + "}=node1 >compound {word:" + nodesToMerge.get(0) + "}=node2)";
    System.out.println("Semgrex Pattern: " + patternString);
    SemgrexPattern semgrexPattern = SemgrexPattern.compile(patternString);
    SemgrexMatcher matcher = semgrexPattern.matcher(graph);

    if (matcher.find()) {
        IndexedWord gov = matcher.getNode("gov");
        IndexedWord node1 = matcher.getNode("node1");
        IndexedWord node2 = matcher.getNode("node2");

        System.out.println("Current edges in the graph:");
        for (SemanticGraphEdge edge : graph.edgeListSorted()) {
            System.out.println(edge);
        }

        // Create the new merged node with combined words "ice" and "cream"
        IndexedWord mergedNode = new IndexedWord(node1);
        mergedNode.setWord(node1.word() + node2.word());
        mergedNode.setLemma(node1.lemma() + node2.lemma());
        mergedNode.setTag("NN");
        mergedNode.setValue(node1.value() + node2.value());

        // Add the merged node to the graph
        graph.addVertex(mergedNode);

        // Create a new edge between the governor and the merged node
        SemanticGraphEdge newEdge = new SemanticGraphEdge(gov, mergedNode, GrammaticalRelation.valueOf("obj"), 1.0, false);
        graph.addEdge(newEdge);

        // Transfer any edges from node1 or node2 to mergedNode (if there are other dependents)
        for (SemanticGraphEdge edge : graph.getOutEdgesSorted(node1)) {
            graph.addEdge(new SemanticGraphEdge(mergedNode, edge.getDependent(), edge.getRelation(), edge.getWeight(), edge.isExtra()));
        }

        for (SemanticGraphEdge edge : graph.getOutEdgesSorted(node2)) {
            graph.addEdge(new SemanticGraphEdge(mergedNode, edge.getDependent(), edge.getRelation(), edge.getWeight(), edge.isExtra()));
        }

        // Print the graph state after adding the merged node and transferring dependencies
        System.out.println("Graph after adding merged node and transferring dependencies:");
        System.out.println(graph.toString(SemanticGraph.OutputFormat.LIST));

        // Disconnect the old nodes instead of removing them fully
        graph.removeEdge(graph.getEdge(gov, node1));
        graph.removeEdge(graph.getEdge(node1, node2));
        
        // Instead of removing the nodes, we mark them as inactive by setting their value to null
        for (IndexedWord node : graph.vertexListSorted()) {
            if (node.equals(node1) || node.equals(node2)) {
                node.setValue(null);  // Mark as disconnected or inactive
            }
        }

        // Check if merged node is retained after disconnection
        if (graph.containsVertex(mergedNode)) {
            System.out.println("Merged node 'creamice' is still in the graph.");
        } else {
            System.out.println("Merged node 'creamice' is missing.");
        }

        // Final graph check
        System.out.println("Final graph after disconnecting old nodes:");
        System.out.println(graph.toString(SemanticGraph.OutputFormat.LIST));

        System.out.println("Manual merge applied successfully.");
    } else {
        System.out.println("No match found for manual merge.");
    }
}

}

'''

with open('SsurgeonMergeNodesComparison.java', 'w') as f:
    f.write(java_code)

# Compile the Java code
compile_command = ["javac", "-encoding", "UTF-8", "-cp", ".:/kaggle/working/stanford-corenlp-4.5.5/*", "SsurgeonMergeNodesComparison.java"]
compile_result = subprocess.run(compile_command, capture_output=True, text=True)

if compile_result.returncode == 0:
    print("Compilation successful")
    
    # Run the Java program
    run_command = ["java", "-cp", ".:/kaggle/working/stanford-corenlp-4.5.5/*", "SsurgeonMergeNodesComparison"]
    run_result = subprocess.run(run_command, capture_output=True, text=True)
    
    print("Program output:")
    print(run_result.stdout)
    
    if run_result.stderr:
        print("Errors or warnings:")
        print(run_result.stderr)
else:
    print("Compilation failed:")
    print(compile_result.stderr)

Compilation successful
Program output:
Original Dependency Graph:
root(ROOT-0, likes-3)
det(child-2, The-1)
nsubj(likes-3, child-2)
compound(cream-5, ice-4)
obj(likes-3, cream-5)
punct(likes-3, .-6)

Testing Ssurgeon MergeNodes:
Graph before applying Ssurgeon MergeNodes:
root(ROOT-0, likes-3)
det(child-2, The-1)
nsubj(likes-3, child-2)
compound(cream-5, ice-4)
obj(likes-3, cream-5)
punct(likes-3, .-6)

Semgrex Pattern: {}=gov >obj ({word:cream}=node1 >compound {word:ice}=node2)
Match found:
Governor node: likes/VBZ
Node1 (cream): cream/NN
Node2 (ice): ice/NN
Graph state before Ssurgeon merge:
root(ROOT-0, likes-3)
det(child-2, The-1)
nsubj(likes-3, child-2)
compound(cream-5, ice-4)
obj(likes-3, cream-5)
punct(likes-3, .-6)

Graph after applying Ssurgeon MergeNodes (before cleanup):
root(ROOT-0, likes-3)
det(child-2, The-1)
nsubj(likes-3, child-2)
compound(cream-5, ice-4)
obj(likes-3, cream-5)
punct(likes-3, .-6)

Ssurgeon MergeNodes applied successfully.
Merged node: cream/NN
Merged no

In [ ]:
#Key observations:
# The original dependency graph correctly represents the sentence structure
# The Semgrex pattern successfully matches the target nodes ("ice" and "cream")
# The Ssurgeon MergeNodes operation reports to be applied successfully
# Despite this, the graph structure remains unchanged after Ssurgeon MergeNodes
# The manual merge successfully creates a new "creamice" node
# The manual merge method properly disconnects the old nodes
# The final graphs for Ssurgeon and manual merges are different

In [ ]:
# Inferences from the Output:
# The Ssurgeon method seems to preserve all existing edges, even after claiming to merge nodes
# Despite the MergeNodes operation, the original nodes ("ice" and "cream") persist in the graph
# The appearance of "null" nodes in both Ssurgeon and manual merge results indicates a potential issue with node removal or deactivation in the graph structure

In [ ]:
# SsurgeonMergeNodes: Attempts to merge "ice" and "cream" using Ssurgeon
# Sets the "pos" (part of speech) to "NN" (noun) for the merged word
# Uses the evaluate method to apply the merge operation directly
# Takes the graph and a SemgrexMatcher as parameters
# Returns a boolean indicating whether the merge was successful

In [ ]:
# Semgrex pattern:
# {}=gov: Matches any word in the sentence and labels it as the governor (gov). In the sentence "The child likes ice cream.", this would match the verb "likes".
# >obj: Indicates that the governor (gov, i.e., "likes") has an object relation (obj) to another word.
# {word:cream}=node1: Matches the word "cream" and labels it as node1.
# >compound {word:ice}=node2: Indicates that node1 ("cream") has a compound relation (compound) to the word "ice", labeled as node2.

In [7]:
import subprocess
import os

java_code = '''
import edu.stanford.nlp.pipeline.*;
import edu.stanford.nlp.semgraph.*;
import edu.stanford.nlp.semgraph.semgrex.*;
import edu.stanford.nlp.ling.*;
import edu.stanford.nlp.util.*;
import edu.stanford.nlp.semgraph.semgrex.ssurgeon.*;

import java.util.*;

public class SsurgeonMergeNodes {

    public static void main(String[] args) {
        Properties props = new Properties();
        props.setProperty("annotators", "tokenize,ssplit,pos,lemma,depparse");
        StanfordCoreNLP pipeline = new StanfordCoreNLP(props);
        
        String text = "The child likes ice cream.";
        
        CoreDocument document = new CoreDocument(text);
        pipeline.annotate(document);
        
        SemanticGraph graph = document.sentences().get(0).dependencyParse();
        
        System.out.println("Original Dependency Graph:");
        System.out.println(graph.toString(SemanticGraph.OutputFormat.LIST));

        testSsurgeonMergeNodes(graph);
    }
    
    private static void testSsurgeonMergeNodes(SemanticGraph graph) {
        System.out.println("Testing Ssurgeon MergeNodes:");
        
        String patternString = "{}=gov >obj ({word:cream}=node1 >compound {word:ice}=node2)";
        System.out.println("Semgrex Pattern: " + patternString);
        SemgrexPattern semgrexPattern = SemgrexPattern.compile(patternString);
        
        SemgrexMatcher matcher = semgrexPattern.matcher(graph);

        if (matcher.find()) {
            IndexedWord gov = matcher.getNode("gov");
            IndexedWord node1 = matcher.getNode("node1");  // "cream"
            IndexedWord node2 = matcher.getNode("node2");  // "ice"
            
            System.out.println("Match found:");
            System.out.println("Governor node: " + gov);
            System.out.println("Node1 (cream): " + node1);
            System.out.println("Node2 (ice): " + node2);

            // Create the Ssurgeon edit for merging nodes
            SsurgeonPattern surgeonPattern = new SsurgeonPattern(semgrexPattern);
            Map<String, String> attributes = new HashMap<>();
            attributes.put("pos", "NN");
            SsurgeonEdit mergeNodesEdit = new MergeNodes(Arrays.asList(node1.word(), node2.word()), attributes);
            surgeonPattern.addEdit(mergeNodesEdit);
            
            // Apply the Ssurgeon merge operation
            boolean editApplied = mergeNodesEdit.evaluate(graph, matcher);
            
            System.out.println("Ssurgeon MergeNodes edit applied: " + editApplied);
            
            System.out.println("Graph after applying Ssurgeon MergeNodes:");
            System.out.println(graph.toString(SemanticGraph.OutputFormat.LIST));
            
            // Check for the merged node
            IndexedWord mergedNode = graph.getNodeByIndex(node1.index());
            if (mergedNode != null && mergedNode.word().equals(node1.word() + node2.word())) {
                System.out.println("Merged node '" + mergedNode.word() + "' is present in the graph.");
                System.out.println("Merged node details: " + mergedNode);
                
                System.out.println("Edges of the merged node:");
                for (SemanticGraphEdge edge : graph.outgoingEdgeList(mergedNode)) {
                    System.out.println("Outgoing: " + edge);
                }
                for (SemanticGraphEdge edge : graph.incomingEdgeList(mergedNode)) {
                    System.out.println("Incoming: " + edge);
                }
            } else {
                System.out.println("Merged node '" + node1.word() + node2.word() + "' is missing or incorrect.");
                System.out.println("Node at index " + node1.index() + ": " + mergedNode);
            }

        } else {
            System.out.println("No match found for the pattern in Ssurgeon.");
        }
    }
}

'''

with open('SsurgeonMergeNodes.java', 'w') as f:
    f.write(java_code)

# Compile the Java code
compile_command = ["javac", "-encoding", "UTF-8", "-cp", ".:/kaggle/working/stanford-corenlp-4.5.5/*", "SsurgeonMergeNodes.java"]
compile_result = subprocess.run(compile_command, capture_output=True, text=True)

if compile_result.returncode == 0:
    print("Compilation successful")
    
    # Run the Java program
    run_command = ["java", "-cp", ".:/kaggle/working/stanford-corenlp-4.5.5/*", "SsurgeonMergeNodes"]
    run_result = subprocess.run(run_command, capture_output=True, text=True)
    
    print("Program output:")
    print(run_result.stdout)
    
    if run_result.stderr:
        print("Errors or warnings:")
        print(run_result.stderr)
else:
    print("Compilation failed:")
    print(compile_result.stderr)

Compilation successful
Program output:
Original Dependency Graph:
root(ROOT-0, likes-3)
det(child-2, The-1)
nsubj(likes-3, child-2)
compound(cream-5, ice-4)
obj(likes-3, cream-5)
punct(likes-3, .-6)

Testing Ssurgeon MergeNodes:
Semgrex Pattern: {}=gov >obj ({word:cream}=node1 >compound {word:ice}=node2)
Match found:
Governor node: likes/VBZ
Node1 (cream): cream/NN
Node2 (ice): ice/NN
Ssurgeon MergeNodes edit applied: false
Graph after applying Ssurgeon MergeNodes:
root(ROOT-0, likes-3)
det(child-2, The-1)
nsubj(likes-3, child-2)
compound(cream-5, ice-4)
obj(likes-3, cream-5)
punct(likes-3, .-6)

Merged node 'creamice' is missing or incorrect.
Node at index 5: cream/NN

Errors or warnings:
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[main] INFO edu.stanford.nlp.tagger.maxent.MaxentTagger - Loading POS tagger from edu/stanford/nlp/models/pos-tagger/english-left3words-distsi

In [ ]:
# Key Observations:
# The original dependency graph correctly represents the sentence "The child likes ice cream."
# The Semgrex pattern successfully matches the compound noun structure "ice cream" in the sentence.
# The MergeNodes operation reports that it was not applied successfully (editApplied: false).
# The dependency graph remains unchanged after the attempted MergeNodes operation.
# The program fails to find a merged node 'creamice' in the graph.
# The node at index 5 remains as "cream/NN", indicating no merging occurred.

In [ ]:
# Key Inferences:
# The Ssurgeon MergeNodes operation is not functioning as expected, despite correctly identifying the nodes to be merged.